In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from torch import nn


In [2]:
! nvidia-smi # check GPU status

Sun Jan  5 12:21:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.73       Driver Version: 410.73       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 29%   36C    P8    15W / 250W |  10532MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 57%   75C    P2   177W / 250W |   7581MiB / 11178MiB |     86%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
# pytorch reads this environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # options: 0, 1, 2, 3

In [4]:
data_root ={
    "train":"train",
    "valid":"valid",
    "test":"test"    
}

In [5]:
torch.utils.data.Dataset

torch.utils.data.dataset.Dataset

In [6]:
train_dataset = ImageFolder(data_root["train"],
                           transform=transforms.ToTensor())

In [7]:
train_dataloader= DataLoader(train_dataset, batch_size=64,shuffle=True)

In [8]:
i=iter(train_dataloader)

In [9]:
X,Y=next(iter(train_dataloader)) #Iterator读取了第一批64个数据

In [10]:
Y

tensor([1, 8, 1, 6, 4, 8, 7, 0, 0, 6, 4, 3, 1, 2, 5, 1, 4, 2, 1, 3, 2, 5, 1, 5,
        1, 4, 5, 4, 1, 4, 0, 5, 1, 3, 0, 2, 7, 7, 9, 4, 0, 9, 4, 9, 0, 7, 1, 7,
        5, 2, 7, 4, 9, 5, 9, 4, 0, 0, 3, 4, 0, 9, 2, 1])

In [11]:
#model = torchvision.models.vgg11(pretrained=True)
model = torch.load("\mynet.pkl")

In [12]:
model.classifier = nn.Sequential(
nn.Linear(in_features=25088, out_features=512, bias=True),
nn.ReLU(inplace=True),
nn.Dropout(p=0.5, inplace=False),
nn.Linear(in_features=512, out_features=512, bias=True),
nn.ReLU(inplace=True),
nn.Dropout(p=0.5, inplace=False),

nn.Linear(in_features=512, out_features=10, bias=True)
)

In [13]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=512, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=512, out_features=10, bias=True)
)

In [14]:
model = model.cuda()

In [15]:
# Y_pred = model(X)
# acc =float(torch.sum(torch.argmax(Y_pred, 1) ==Y )) / float(len(Y))
# acc

In [16]:
opt = torch.optim.SGD(model.parameters(),1e-3)

In [17]:
loss = nn.CrossEntropyLoss()

In [18]:
#loss(model(X),Y)

In [19]:
for epoch in range(20):
    print(f"--------epoch {epoch+1}--------")
    for (iternum, (X,Y)) in enumerate(train_dataloader,1):
    #for (X,Y) in train_dataloader:
    #iteration
        X,Y=X.cuda(),Y.cuda()
    
        Y_pred = model(X)
        l= loss(Y_pred,Y)
    
        opt.zero_grad()
        l.backward()
        opt.step()
        #print('hah')
        #print(iternum)
        #logging
        if iternum%200 == 0:
            acc =float(torch.sum(torch.argmax(Y_pred, 1) ==Y )) / float(len(Y))
            #print(iternum)
            print(f"Iter {iternum}:loss {l:.6f} accuarcy {acc:.4f}")
            #print(acc)
    

--------epoch 1--------
Iter 200:loss 0.957959 accuarcy 0.7031
Iter 400:loss 0.814629 accuarcy 0.7344
Iter 600:loss 0.755053 accuarcy 0.7031
Iter 800:loss 0.552171 accuarcy 0.7812
Iter 1000:loss 0.654004 accuarcy 0.7500
Iter 1200:loss 0.896904 accuarcy 0.7344
Iter 1400:loss 0.446449 accuarcy 0.8281
--------epoch 2--------
Iter 200:loss 0.740757 accuarcy 0.7500
Iter 400:loss 0.518233 accuarcy 0.8125
Iter 600:loss 0.520742 accuarcy 0.7969
Iter 800:loss 0.369287 accuarcy 0.9219
Iter 1000:loss 0.593037 accuarcy 0.7188
Iter 1200:loss 0.392950 accuarcy 0.8594
Iter 1400:loss 0.501656 accuarcy 0.8281
--------epoch 3--------
Iter 200:loss 0.411496 accuarcy 0.8594
Iter 400:loss 0.346921 accuarcy 0.9062
Iter 600:loss 0.806195 accuarcy 0.6875
Iter 800:loss 0.588158 accuarcy 0.7969
Iter 1000:loss 0.711640 accuarcy 0.7656
Iter 1200:loss 0.353300 accuarcy 0.8750
Iter 1400:loss 0.525600 accuarcy 0.8281
--------epoch 4--------
Iter 200:loss 0.542823 accuarcy 0.8281
Iter 400:loss 0.386795 accuarcy 0.843

In [20]:
# for (iternum, (X,Y)) in enumerate(train_dataloader,1):
# #for (X,Y) in train_dataloader:
#     #iteration
#     X,Y=X.cuda(),Y.cuda()
    
#     Y_pred = model(X)
#     l= loss(Y_pred,Y)
    
#     opt.zero_grad()
#     l.backward()
#     opt.step()
#     #print('hah')
#     #print(iternum)
#     #logging
#     if iternum%200 == 0:
#         acc =float(torch.sum(torch.argmax(Y_pred, 1) ==Y )) / float(len(Y))
#         #print(iternum)
#         print(acc)

In [21]:
torch.save(model,"\mynet.pkl")

In [22]:
model = torch.load("\mynet.pkl")

In [23]:
# for test usage - do not modify this
import pandas as pd
import numpy as np
import math
import os

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from more_itertools import chunked
from tqdm import tqdm_notebook

In [24]:
class TestData(ImageFolder):
    def __getitem__(self, idx):
        # drop labels
        return super(TestData, self).__getitem__(idx)[0]

In [25]:
test_datafolder = TestData(data_root["test"], transform=transforms.ToTensor())
classes = np.array(test_datafolder.classes)
imgs = [x for (x, _) in test_datafolder.imgs]
labels = classes[[x for (_, x) in test_datafolder.imgs]]

In [26]:
# load all test images from disk(SSD) into memory
# this is expected to take minutes
# this may fails due to system kill signal - just try it again later
testdata = next(iter(DataLoader(test_datafolder, batch_size=len(test_datafolder))))
assert testdata.shape == torch.Size([90000, 3, 32, 32])

In [27]:
model = model.cuda()
testdata = testdata.cuda()

In [28]:
def evaluate(model, testdata, imgs, labels, batch_size=512):
    files_loader = [x for x in chunked(imgs, batch_size)]
    test_dataloader = DataLoader(testdata, batch_size=batch_size)
    groundtruth_loader = chunked(labels, batch_size)
    num_batches = len(files_loader)

    columns = ["file", "prediction", "ground_truth"]
    data_iter = iter(zip(test_dataloader, files_loader, groundtruth_loader))
    df = pd.DataFrame(columns=columns)
    for _ in tqdm_notebook(range(num_batches)):
        X, filepath, Y = next(data_iter)
        with torch.no_grad():
            idx = torch.argmax(model(X), 1).cpu()
        prediction = classes[idx]

        buffer = pd.DataFrame(zip(filepath, prediction, Y), columns=columns)
        df = df.append(buffer)

    return df

In [29]:
df = evaluate(model, testdata, imgs, labels)
df

,file,prediction,ground_truth
0,test/airplane/cifar10-test-10.png,dog,airplane
1,test/airplane/cifar10-test-1001.png,airplane,airplane
2,test/airplane/cifar10-test-1010.png,airplane,airplane
3,test/airplane/cifar10-test-1018.png,airplane,airplane
4,test/airplane/cifar10-test-1022.png,airplane,airplane
...,...,...,...
395,test/truck/n04520170_9909.png,truck,truck
396,test/truck/n04520170_9936.png,truck,truck
397,test/truck/n04520170_9942.png,automobile,truck
398,test/truck/n04520170_9963.png,truck,truck


In [30]:
accu = np.sum(df["ground_truth"] == df["prediction"])/float(len(df)) * 100
print(f"Accuracy {accu:.3f}%")

Accuracy 71.303%


In [31]:
df.to_csv("test.csv", index=False)

In [32]:
exit()

在实验中发现，当epoch取足够大时，训练集的准确率能达到100%，但是在测试集的准确率却并没有提高多少。可考虑，增加网络深度等来提高准确率。

在实验刚开始时，是用的torchvision.models.vgg11(pretrained=True)，后来将训练好的保存在mynet.pkl中，后来直接调用。